# The NumPy array

## A structure for efficient numerical computation

<span style="font-size: 140%; line-height: 1.3em;">
Stéfan van der Walt (@stefanvdwalt)<br/>
University of California, Berkeley<br/>
Advanced Scientific Programming in Python
</span>

<span style="font-size: 130%; text-align: right;">
Summer School 2016 held in Reading, UK
</span>

Accompanying **problem sets** are on the Wiki at:
 https://python.g-node.org/wiki/numpy

<img src="images/overview.png" width="80%"/>

## Num-what?

This talk discusses some of the more advanced NumPy features. If you’ve
never seen NumPy before, you may have more fun doing the tutorial at

http://mentat.za.net/numpy/intro/intro.html

or studying the course notes at

http://scipy-lectures.github.io

You can always catch up by reading:

>  **The NumPy array: a structure for efficient numerical computation**. Stéfan
>  van der Walt, S. Chris Colbert and Gaël Varoquaux. In IEEE Computing in
>  Science Engineering, March/April 2011.

Nicolas Rougier also has a beautiful tutorial at

https://github.com/rougier/numpy-tutorial

## Setup

In [ ]:
import numpy as np  # We always use this convention,
                    # also in the documentation

## The NumPy ndarray

### Revision: Structure of an array

<img src="images/array_memory_presentation.png" width="80%"/>

<img src="images/ndarray_struct.png" width="50%"/>

In [ ]:
x = np.array([[ 0,  1,  2,  3],
              [ 4,  5,  6,  7],
              [ 8,  9, 10, 11]])
x

## A homogeneous container

In [ ]:
x.data

Data is just a pointer to bytes in memory:

<img src="images/array_dtype.png" width="60%">

In [ ]:
def print_info(a):
    print('number of elements:', a.size)
    print('number of dimensions:', a.ndim)
    print('shape:', a.shape)
    print('data type:', a.dtype)
    print('strides:', a.strides)
    print('flags:')
    print(a.flags)
    
print_info(x)

## Dimensions

In [ ]:
x.shape

<img src="images/array_flat_extended.png" width="80%"/>
<br/>
<div style="text-align: center;">
<img style="display: inline;" src="images/array_reshape.png" width="40%"/>
<img style="display: inline;" src="images/array_reshape_nd.png" width="40%"/>
</div>


In [ ]:
x = np.array([])

In [ ]:
np.array(0).shape

In [ ]:
np.array(()).shape

In [ ]:
x = np.random.random((3, 2, 3, 3))
x.shape

In [ ]:
x.ndim

### Common dtypes

In [ ]:
x.dtype

Common types in include ``int``, ``float``, ``bool``:

In [ ]:
np.array ([-1, 0, 1], dtype=int)

In [ ]:
np.array([-1, 0, 1], dtype=float)

In [ ]:
np.array([-1 , 0, 1], dtype=bool)

Each item in the array has to have the same type (occupy a fixed nr of bytes
in memory), but that does not mean a type has to consist of a single item:

In [ ]:
dt = np.dtype([('value', np.int), ('status', np.bool)])
x = np.array([(0, True), (1, False)], dtype=dt)
x

This is called a structured array.

### Strides

In [ ]:
x

In [ ]:
x = np.arange(12).reshape((3, 4))
x

In [ ]:
x.dtype

In [ ]:
x.dtype.itemsize

In [ ]:
x.strides

We see ``(4 * itemsize, itemsize)`` or ``(skip_bytes_row, skip_bytes_col)``.

<img src="images/array_memory_dtype.png" width="100%"/>

In [ ]:
xbytes = x.ravel().view(dtype=np.uint8)
xbytes

In [ ]:
xbytes[:8], xbytes[8:16], xbytes[16:24]

### Example: Taking the transpose

We can take the transpose of the array *without* copying any data by simply manipulating ``shape`` and ``strides``.  What should they be?

<img src="images/array_transpose_question.png" width="100%"/>

In [ ]:
print_info(x.T)

### Flags

In [ ]:
x.flags

In [ ]:
z = x[::2]

z.flags

### Copies

In [ ]:
y = x

y[0, 0] = 5
print(y)
y[0, 0] = 100

In [ ]:
x

In [ ]:
y.flags.owndata

# Advanced operations: axis-wise evaluation

In [ ]:
expr = np.load('expr.npz')['expr']

In [ ]:
expr

In [ ]:
print_info(expr)

This has the raw read count data. However, each sample gets a different number of reads, so we want to normalize by the *library size*, which is the total number of reads across a column.

The `np.sum` function returns the sum of all the elements of an array. With the `axis` argument, you can take the sum *along the given axis*.

In [ ]:
lib_size = np.sum(expr, axis=0)
lib_size.shape

In [ ]:
np.sum(expr, axis=1).shape

### Exercise

Generate a 10 x 3 array of random numbers. From each row, pick the column containing the number closest to 0.75.

*Hint:* use of `np.abs` and `np.argmin` to find the column j that contains the closest element in each row i. The final result should be a vector of integers of shape (10,).

### Exercise

Some applications, such as clustering, are computationally expensive, and wouldn't work without first doing some form of *feature selection*, where we discard most of the data and keep only what we think will be most useful. One simple version is to keep only the genes with the most variance (as these will be more informative than genes that don't vary between patients).

- Find the variance across patients of all the genes (rows) in the expression dataset.
- Use `np.argsort` to find the location of the 1,500 most variable genes.
- Use these indices to produce a shape (1500, 375) matrix containing only the most variable genes.

## Structured arrays

Like we saw earlier, each item in an array has the same type, but that does not
mean a type has to consist of a single item:

In [ ]:
dt = np.dtype([('value', np.int), ('status', np.bool)])
np.array([(0, True), (1, False)], dtype=dt)

This is called a structured array, and is accessed like a dictionary:

In [ ]:
x = np.array([(0, True), (1, False)], dtype=dt)
x['value']

In [ ]:
x['status']

### Reading structured data from file

<img src="images/radar_struct.png" width="100%"/>

Reading this kind of data can be troublesome:

```matlab
while ((count > 0) && (n <= NumPoints))
  % get time - I8 [ ms ]
  [lw, count] = fread(fid, 1, 'uint32');
  if (count > 0) % then carry on
  uw = fread(fid, 1, 'int32');
  t(1, n) = (lw + uw * 2^32) / 1000;

  % get number of bytes of data
  numbytes = fread(fid, 1, 'uint32');

  % read sMEASUREMENTPOSITIONINFO (11 bytes)
  m(1, n) = fread(fid, 1, 'float32'); % az [ rad ]
  m(2, n) = fread(fid, 1, 'float32'); % el [ rad ]
  m(3, n) = fread(fid, 1, 'uint8');   % region type
  m(4, n) = fread(fid, 1, 'uint16');  % region ID
  g(1, n) = fread(fid, 1, 'uint8');

  numsamples = ( numbytes -12)/2; % 2 byte int
  ...
```

Using structured arrays:

```python
dt = np.dtype([('time', np.uint64),
               ('size', np.uint32),
               ('position', [('az', np.float32),
                             ('el', np.float32),
                             ('region_type', np.uint8),
                             ('region_ID', np.uint16)]),
               ('gain', np.uint8),
               ('samples', (np.int16, 2048))])

data = np.fromfile(f, dtype=dt)
```

We can then access this structured array as before:

```python
data['position']['az']
```

## Broadcasting

Combining of differently shaped arrays without creating large intermediate
arrays:

In [ ]:
x = np.arange(4)
print(x)
print(x + 3)

![Broadcasting in 1D](images/broadcast_1D.png)

See docstring of ``np.doc.broadcasting`` for more detail.

## Broadcasting (2D)

In [ ]:
a = np.arange(12).reshape((3, 4))
b = np.array([1, 2, 3])[:, np.newaxis]

print(a)
print()
print(b)

In [ ]:
a + b

<img src="images/broadcast_2D.png"/>

## Broadcasting (3D)

In [ ]:
x = np.zeros((3, 5))
y = np.zeros(8)
z = x[..., np.newaxis]

```python
x = np.zeros((3, 5))
y = np.zeros(8)
z = x[..., np.newaxis]
```

```
X shape = (3, 5, 1)
Y shape = (      8)
Z shape = (3, 5, 8)
```

<img src="images/broadcast_3D.png"/>

## Broadcasting rules

The broadcasting rules are straightforward: compare dimensions, starting from
the last. Match when either dimension is one or ``None``, or if dimensions are
equal:

```
Scalar    2D           3D           Bad

( ,)     (3, 4)     (3, 5, 1)    (3, 5, 2)
(3,)     (3, 1)     (      8)    (      8)
----     ------     ---------    ---------
(3,)     (3, 4)     (3, 5, 8)       XXX
```

## Explicit broadcasting

In [ ]:
x = np.zeros((3, 5, 1))
y = np.zeros((3, 5, 8))

xx, yy = np.broadcast_arrays(x, y)

xx.shape

```python
np.broadcast_arrays([1, 2, 3],
                    [[1], [2], [3]])
```

```
[[1 2 3]   [[1 1 1]
 [1 2 3]    [2 2 2]
 [1 2 3]]   [3 3 3]]
```

In [ ]:
np.broadcast_to(x, (3,5,8)).shape

## Fancy indexing

An ndarray can be indexed in two ways:

 - Using slices and scalars
 - Using other ndarrays ("fancy indexing")

In [ ]:
x = np.arange(9).reshape((3, 3))
x

In [ ]:
x[:, [1, 1, 2]]

Which is equivalent to:

In [ ]:
np.array((x[:, 1], x[:, 1], x[:, 2])).T

### Output shape of an indexing op

1. Broadcast all index arrays against one another.
2. Use the dimensions of slices as-is.

Start with the following input:

In [ ]:
x = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])

In [ ]:
x.shape

What would happen if we do

In [ ]:
idx0 = np.array([[0, 1],
                 [1, 2]])  # row indices

idx1 = np.array([[0, 1]])  # column indices

```python
x[idx0, idx1]
```

### Output shape of indexing op (cont'd)

In [ ]:
idx0.shape, idx1.shape

In [ ]:
a, b = np.broadcast_arrays(idx0, idx1)
a

In [ ]:
b

In [ ]:
x

In [ ]:
x[idx0, idx1]


## Output shape of an indexing op (cont'd)

In [ ]:
x = np.random.random((15, 12, 16, 3))

index_one = np.array([[0, 1], [2, 3], [4, 5]])
index_two = np.array([[0, 1]])

In [ ]:
print(index_one.shape)
print(index_two.shape)

Predict the output shape of

```python
x[5:10, index_one, :, index_two]
```

Warning! When mixing slicing and fancy indexing, the
*order* of the output dimensions is less easy to predict.
Play it safe and **don't mix the two!**

## Output shape of an indexing op (cont'd)

In [ ]:
x = np.random.random((15, 12, 16, 3))
index_one = np.array([[0, 1], [2, 3], [4, 5]])
index_two = np.array([[0, 1]])

In [ ]:
print(index_one.shape)
print(index_two.shape)

Broadcast ``index1`` against ``index2``:

```
(3, 2)  # shape of index1
(1, 2)  # shape of index2
------
(3, 2)
```

The shape of ``x[5:10, index_one, :, index_two]`` is

``(3, 2, 5, 16)``

## Jack's dilemma

Indexing and broadcasting are intertwined, as we’ll see in the following
example. One of my favourites from the NumPy mailing list:

```email
Date: Wed, 16 Jul 2008 16:45:37 -0500
From: Jack Cook
To: <numpy-discussion@scipy.org>
Subject: Numpy Advanced Indexing Question
```

Greetings,

I have an I,J,K 3D volume of amplitude values at regularly sampled
time intervals. I have an I,J 2D slice which contains a time (K)
value at each I, J location. What I would like to do is extract a
subvolume at a constant +/- K window around the slice. Is there an
easy way to do this using advanced indexing or some other method?
Thanks in advanced for your help.

-- Jack

<img src="images/jack.png" width="90%" alt="Jack's cube"/>

### Test setup

In [ ]:
ni, nj, nk = (10, 15, 20)  # dimensions
half_width = 3  # take 7 elements

Make a fake data block such that ``block[i, j, k] == k`` for all i, j, k.

In [ ]:
block = np.empty((ni, nj, nk) , dtype=int)
block[:] = np.arange(nk)[np.newaxis, np.newaxis, :]

Pick out a random fake horizon in ``k``:

In [ ]:
rng = np.random.RandomState(0)

k = rng.randint(5, 15, size=(ni, nj))
k

### Solution

These two indices assure that we take a slice at each (i, j) position

In [ ]:
idx_i = np.arange(ni)[:, np.newaxis, np.newaxis]
idx_j = np.arange(nj)[np.newaxis, :, np.newaxis]

This is the substantitive part that picks out the window:

In [ ]:
idx_k = k[:, :, np.newaxis] + np.arange(-half_width, half_width + 1)
slices = block[idx_i, idx_j, idx_k]  # Slice

Apply the broadcasting rules:

```
(ni,  1, 1                 )  # idx_i
(1,  nj, 1                 )  # idx_j
(ni, nj, 2 * half_width + 1)  # idx_k
-
(ni, nj, 7)  <-- this is what we wanted!
```

### Solution verification

In [ ]:
slices = block[idx_i, idx_j, idx_k]
slices.shape
(10, 15, 7)

Now verify that our window is centered on ``k`` everywhere:

In [ ]:
slices[:, :, 3]

In [ ]:
np.all(slices[:, :, 3] == k)

## The \_\_array\_interface\_\_

Any object that exposes a suitable dictionary named
``__array_interface__`` may be converted to a NumPy array. This is
handy for exchanging data with external libraries. The array interface
has the following important keys (see
http://docs.scipy.org/doc/numpy/reference/arrays.interface):

 - **shape**
 - **typestr**
 - **data**: (20495857, True); 2-tuple—pointer to data and boolean to
indicate whether memory is read-only
 - **strides**
 - **version**: 3

## Stride Tricks: Repeating sequence

In [ ]:
data = np.array([1, 2, 3, 4])
print(data.shape, data.strides)

In [ ]:
N = 400000
repeat = np.lib.stride_tricks.as_strided(data, shape=(N, 4), strides=(0, 8))

In [ ]:
repeat

# Did you know?

NumPy implements a Diophantine (integer) [equation solver](https://github.com/numpy/numpy/blob/master/numpy/core/src/private/mem_overlap.c) for `np.shares_memory` to calculate whether two arrays possibly overlap in memory.

Q: Can you imagine why this is a tricky problem to solve?

```
sum(a[i] * x[i] for i in range(n)) == b

a[i] > 0, 0 <= x[i] <= ub[i]
```

You can even access this solver from Python.  For example, given that a year has 365 days, how do we fit in months of 28, 30, and 31 days?

In [ ]:
from numpy.core.multiarray_tests import solve_diophantine

month_days = (28, 30, 31)
limits = (20, 20, 20)

solve_diophantine(month_days, limits, 365)

In [ ]:
(2 * 28) + (1 * 30) + (9 * 31) 

In [ ]:
solve_diophantine(month_days, (1, 20, 20), 365)

In [ ]:
(1 * 28) + (4 * 30) + (7 * 31)

## Questions, discussion and exercises

---
<div style="height: 200px;"/>

In [ ]:
%reload_ext load_style
%load_style css/notebook.css